# Tensorflow Road Signs YOLO Demo

>⚠️ **Warning:** This demo assumes that you have access to an on-prem deployment of Dioptra that provides a copy of the Road Signs dataset and a CUDA-compatible GPU.
> This demo cannot be run on a typical personal computer.

The demo provided in the Jupyter notebook `demo.ipynb` contains an example of how to set up and train a model based on the YOLO v1 architecture and use it to perform object detection on the Road Signs dataset.

## Setup

In [1]:
# Import packages from the Python standard library
import os
import pprint
import random
import time
import warnings
from pathlib import Path
from typing import Tuple

# Please enter custom username here.
USERNAME = "jglasbrenner"

# Filter out warning messages
warnings.filterwarnings("ignore")

# Address for connecting the docker container to exposed ports on the host device
HOST_DOCKER_INTERNAL = "host.docker.internal"
# HOST_DOCKER_INTERNAL = "172.17.0.1"

# Testbed API ports
RESTAPI_PORT = "20080"
MLFLOW_TRACKING_PORT = "25000"

# Default address for accessing the RESTful API service
RESTAPI_ADDRESS = (
    # f"http://{HOST_DOCKER_INTERNAL}:{RESTAPI_PORT}"
    f"http://restapi:{RESTAPI_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{RESTAPI_PORT}"
)

# Override the AI_RESTAPI_URI variable, used to connect to RESTful API service
os.environ["AI_RESTAPI_URI"] = RESTAPI_ADDRESS

# Default address for accessing the MLFlow Tracking server
MLFLOW_TRACKING_URI = (
    # f"http://{HOST_DOCKER_INTERNAL}:{MLFLOW_TRACKING_PORT}"
    f"http://mlflow-tracking:{MLFLOW_TRACKING_PORT}"
    if os.getenv("IS_JUPYTER_SERVICE")
    else f"http://localhost:{MLFLOW_TRACKING_PORT}"
)

# Override the MLFLOW_TRACKING_URI variable, used to connect to MLFlow Tracking service
os.environ["MLFLOW_TRACKING_URI"] = MLFLOW_TRACKING_URI

# Path to custom task plugins archives
CUSTOM_PLUGINS_BACKEND_CONFIGS_TAR_GZ = Path("custom-plugins-backend-configs.tar.gz")
CUSTOM_PLUGINS_EVALUATION_TAR_GZ = Path("custom-plugins-evaluation.tar.gz")
CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ = Path("custom-plugins-roadsigns-yolo-estimators.tar.gz")
CUSTOM_PLUGINS_TRACKING_TAR_GZ = Path("custom-plugins-tracking.tar.gz")

# Base API address
RESTAPI_API_BASE = f"{RESTAPI_ADDRESS}/api"

# Path to workflows archive
WORKFLOWS_TAR_GZ = Path("workflows.tar.gz")

# Experiment name (note the username_ prefix convention)
EXPERIMENT_NAME = f"{USERNAME}_roadsigns_yolo"

# Import third-party Python packages
import numpy as np
import requests
from mlflow.tracking import MlflowClient

# Import utils.py file
import utils

# Create random number generator
rng = np.random.default_rng(54399264723942495723666216079516778448)

## Dataset

The Road Signs dataset needed for this demo can be obtained here: https://makeml.app/datasets/road-signs.
Object bounding boxes are provided in the PascalVOC format, which organizes the dataset into two folders:

    annotations/  (xml files)
    images/       (png files)

The PascalVOC format uses filenames to associate images with their corresponding annotation.
For example, the image file `images/road1.png` has an associated annotation file `annotations/road1.xml`.

This dataset does not provide a testing dataset, so we need to create our own.
We do this by stratifying over the number of objects and classes in the images and sampling 10% of the images without replacement within each of these groups.
The exact train/test split we used is reported in the `roadsigns_train_test_split.csv` file.

After performing our train/test split, our data is reorganized into the following folder structure:

    roadsigns
    ├── testing
    │   ├── annotations
    │   └── images
    └── training
        ├── annotations
        └── images

## Submit and run jobs

The entrypoints that we will be running in this example are implemented in the Python source files under `src/` and the `MLproject` file.
To run these entrypoints within the testbed architecture, we need to package those files up into an archive and submit it to the Testbed RESTful API to create a new job.
For convenience, the `Makefile` provides a rule for creating the archive file for this example, just run `make workflows`,

In [ ]:
%%bash

# Create the workflows.tar.gz file
make workflows

To connect with the endpoint, we will use a client class defined in the `utils.py` file that is able to connect with the Testbed RESTful API using the HTTP protocol.
We connect using the client below, which uses the environment variable `AI_RESTAPI_URI` to figure out how to connect to the Testbed RESTful API,

In [17]:
restapi_client = utils.SecuringAIClient()

We need to register an experiment under which to collect our job runs.
The code below checks if the relevant experiment exists.
If it does, then it just returns info about the experiment, if it doesn't, it then registers the new experiment.

In [ ]:
response_experiment = restapi_client.get_experiment_by_name(name=EXPERIMENT_NAME)

if response_experiment is None or "Not Found" in response_experiment.get("message", []):
    response_experiment = restapi_client.register_experiment(name=EXPERIMENT_NAME)

response_experiment

We should also check which queues are available for running our jobs to make sure that the resources that we need are available.
The code below queries the Testbed API and returns a list of active queues.

In [ ]:
restapi_client.list_queues()

In [ ]:
response_queue = restapi_client.get_queue_by_name(name="tensorflow_gpu")

if response_queue is None or "Not Found" in response_queue.get("message", []):
    response_queue = restapi_client.register_queue(name="tensorflow_gpu")

response_queue

This example also makes use of the `evaluation` and `roadsigns_yolo_estimators` packages stored locally under the `task-plugins/securingai_custom` directory.
To register these custom task plugins, we first need to package them up into an archive.
For convenience, the `Makefile` provides a rule for creating the custom task plugins archive file, just run `make custom-plugins`,

In [ ]:
# Delete the 'backend_configs' custom task plugin package
restapi_client.delete_custom_task_plugin(name="backend_configs")

# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")

# Delete the `roadsigns_yolo_estimators` package
restapi_client.delete_custom_task_plugin(name="roadsigns_yolo_estimators")

# Delete the 'tracking' custom task plugin package
restapi_client.delete_custom_task_plugin(name="tracking")

In [ ]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins

Now that the custom task plugin packages are packaged into archive files, next we register them by uploading the files to the REST API.
Note that we need to provide the name to use for custom task plugin package, and this name must be unique under the custom task plugins namespace.
For a full list of the custom task plugins, use `restapi_client.restapi_client.list_custom_task_plugins()`.

In [ ]:
response_backend_configs_custom_plugins = restapi_client.get_custom_task_plugin(name="backend_configs")

if response_backend_configs_custom_plugins is None or "Not Found" in response_backend_configs_custom_plugins.get("message", []):
    response_backend_configs_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="backend_configs",
        custom_plugin_file=CUSTOM_PLUGINS_BACKEND_CONFIGS_TAR_GZ,
    )

response_backend_configs_custom_plugins

In [ ]:
response_evaluation_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")

if response_evaluation_custom_plugins is None or "Not Found" in response_evaluation_custom_plugins.get("message", []):
    response_evaluation_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

response_evaluation_custom_plugins

In [ ]:
response_roadsigns_custom_plugins = restapi_client.get_custom_task_plugin(name="roadsigns_yolo_estimators")

if response_roadsigns_custom_plugins is None or "Not Found" in response_roadsigns_custom_plugins.get("message", []):
    response_roadsigns_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="roadsigns_yolo_estimators",
        custom_plugin_file=CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ,
    )

response_roadsigns_custom_plugins

In [ ]:
response_tracking_custom_plugins = restapi_client.get_custom_task_plugin(name="tracking")

if response_tracking_custom_plugins is None or "Not Found" in response_tracking_custom_plugins.get("message", []):
    response_tracking_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="tracking",
        custom_plugin_file=CUSTOM_PLUGINS_TRACKING_TAR_GZ,
    )

response_tracking_custom_plugins

If at any point you need to update one or more files within the `backend_configs`, `evaluation`, or `roadsigns_yolo_estimators` packages, you will need to unregister/delete the custom task plugin first using the REST API.
This can be done as follows,

```python
# Delete the 'backend_configs' custom task plugin package
restapi_client.delete_custom_task_plugin(name="backend_configs")

# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")

# Delete the `roadsigns_yolo_estimators` package
restapi_client.delete_custom_task_plugin(name="roadsigns_yolo_estimators")

# Delete the `tracking` package
restapi_client.delete_custom_task_plugin(name="tracking")
```

In [ ]:
# Delete the 'backend_configs' custom task plugin package
restapi_client.delete_custom_task_plugin(name="backend_configs")

# Delete the 'evaluation' custom task plugin package
restapi_client.delete_custom_task_plugin(name="evaluation")

# Delete the `roadsigns_yolo_estimators` package
restapi_client.delete_custom_task_plugin(name="roadsigns_yolo_estimators")

# Delete the `tracking` package
restapi_client.delete_custom_task_plugin(name="tracking")

After you have deleted the task plugin in the testbed, re-run the `make custom-plugins` code block to update the package archive, then upload the updated plugin by re-running the `restapi_client.upload_custom_plugin_packge` block.

Next, we need to use transfer learning to update the object detection layers in our model.

```python
# Submit transfer learning job for the mobilenet_v2 + yolo network architecture
response_mobilenet_v2_yolo_transfer_learn = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    queue="tensorflow_gpu",
    entry_point="transfer_learn",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/roadsigns/training",
        "-P model_architecture=mobilenet_v2",
        "-P epochs=300",
        "-P batch_size=32",
        f"-P register_model_name={EXPERIMENT_NAME}_mobilenet_v2_yolo",
    ]),
)

print("Transfer learning job for MobileNet V2 + YOLO neural network submitted")
print("")
pprint.pprint(response_mobilenet_v2_yolo_transfer_learn)
```

In [ ]:
# Submit transfer learning job for the mobilenet_v2 + yolo network architecture
response_mobilenet_v2_yolo_transfer_learn = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    queue="tensorflow_gpu",
    entry_point="transfer_learn",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/roadsigns/training",
        "-P model_architecture=mobilenet_v2",
        "-P epochs=300",
        "-P batch_size=32",
        # f"-P register_model_name={EXPERIMENT_NAME}_mobilenet_v2_yolo",
    ]),
)

print("Transfer learning job for MobileNet V2 + YOLO neural network submitted")
print("")
pprint.pprint(response_mobilenet_v2_yolo_transfer_learn)

In [21]:
# Delete the 'backend_configs' custom task plugin package
pprint.pprint(restapi_client.delete_custom_task_plugin(name="backend_configs"))

# Delete the 'evaluation' custom task plugin package
pprint.pprint(restapi_client.delete_custom_task_plugin(name="evaluation"))

# Delete the `roadsigns_yolo_estimators` package
pprint.pprint(restapi_client.delete_custom_task_plugin(name="roadsigns_yolo_estimators"))

# Delete the `tracking` package
pprint.pprint(restapi_client.delete_custom_task_plugin(name="tracking"))

{'collection': 'securingai_custom',
 'status': 'Success',
 'taskPluginName': ['backend_configs']}
{'collection': 'securingai_custom',
 'status': 'Success',
 'taskPluginName': ['evaluation']}
{'collection': 'securingai_custom',
 'status': 'Success',
 'taskPluginName': ['roadsigns_yolo_estimators']}
{'collection': 'securingai_custom',
 'status': 'Success',
 'taskPluginName': ['tracking']}


In [22]:
%%bash

# Create the workflows.tar.gz file
make custom-plugins workflows

tar -C task-plugins/securingai_custom -czf custom-plugins-roadsigns-yolo-estimators.tar.gz roadsigns_yolo_estimators/__init__.py roadsigns_yolo_estimators/data.py roadsigns_yolo_estimators/feature_extraction_layers.py roadsigns_yolo_estimators/finetuning.py roadsigns_yolo_estimators/keras_classifiers.py roadsigns_yolo_estimators/keras_object_detectors.py roadsigns_yolo_estimators/losses.py roadsigns_yolo_estimators/methods.py roadsigns_yolo_estimators/metrics.py roadsigns_yolo_estimators/output_layers.py roadsigns_yolo_estimators/utils.py
chmod 644 custom-plugins-roadsigns-yolo-estimators.tar.gz
tar  -czf workflows.tar.gz src/infer.py src/transfer_learn.py MLproject
chmod 644 workflows.tar.gz


In [ ]:
response_backend_configs_custom_plugins = restapi_client.get_custom_task_plugin(name="backend_configs")

if response_backend_configs_custom_plugins is None or "Not Found" in response_backend_configs_custom_plugins.get("message", []):
    response_backend_configs_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="backend_configs",
        custom_plugin_file=CUSTOM_PLUGINS_BACKEND_CONFIGS_TAR_GZ,
    )

pprint.pprint(response_backend_configs_custom_plugins)

response_evaluation_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")

if response_evaluation_custom_plugins is None or "Not Found" in response_evaluation_custom_plugins.get("message", []):
    response_evaluation_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

pprint.pprint(response_evaluation_custom_plugins)

response_roadsigns_custom_plugins = restapi_client.get_custom_task_plugin(name="roadsigns_yolo_estimators")

if response_roadsigns_custom_plugins is None or "Not Found" in response_roadsigns_custom_plugins.get("message", []):
    response_roadsigns_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="roadsigns_yolo_estimators",
        custom_plugin_file=CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ,
    )

pprint.pprint(response_roadsigns_custom_plugins)

response_tracking_custom_plugins = restapi_client.get_custom_task_plugin(name="tracking")

if response_tracking_custom_plugins is None or "Not Found" in response_tracking_custom_plugins.get("message", []):
    response_tracking_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="tracking",
        custom_plugin_file=CUSTOM_PLUGINS_TRACKING_TAR_GZ,
    )

pprint.pprint(response_tracking_custom_plugins)

# Submit transfer learning job for the mobilenet_v2 + yolo network architecture
response_mobilenet_v2_yolo_transfer_learn = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    queue="tensorflow_gpu",
    entry_point="transfer_learn",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/roadsigns/training",
        "-P model_architecture=mobilenetv2",
        "-P epochs=300",
        "-P batch_size=32",
        f"-P register_model_name={EXPERIMENT_NAME}_mobilenetv2_yolo",
    ]),
)

print("Transfer learning job for MobileNet V2 + YOLO neural network submitted")
print("")
pprint.pprint(response_mobilenet_v2_yolo_transfer_learn)

In [23]:
response_backend_configs_custom_plugins = restapi_client.get_custom_task_plugin(name="backend_configs")

if response_backend_configs_custom_plugins is None or "Not Found" in response_backend_configs_custom_plugins.get("message", []):
    response_backend_configs_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="backend_configs",
        custom_plugin_file=CUSTOM_PLUGINS_BACKEND_CONFIGS_TAR_GZ,
    )

pprint.pprint(response_backend_configs_custom_plugins)

response_evaluation_custom_plugins = restapi_client.get_custom_task_plugin(name="evaluation")

if response_evaluation_custom_plugins is None or "Not Found" in response_evaluation_custom_plugins.get("message", []):
    response_evaluation_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="evaluation",
        custom_plugin_file=CUSTOM_PLUGINS_EVALUATION_TAR_GZ,
    )

pprint.pprint(response_evaluation_custom_plugins)

response_roadsigns_custom_plugins = restapi_client.get_custom_task_plugin(name="roadsigns_yolo_estimators")

if response_roadsigns_custom_plugins is None or "Not Found" in response_roadsigns_custom_plugins.get("message", []):
    response_roadsigns_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="roadsigns_yolo_estimators",
        custom_plugin_file=CUSTOM_PLUGINS_ROADSIGNS_YOLO_ESTIMATORS_TAR_GZ,
    )

pprint.pprint(response_roadsigns_custom_plugins)

response_tracking_custom_plugins = restapi_client.get_custom_task_plugin(name="tracking")

if response_tracking_custom_plugins is None or "Not Found" in response_tracking_custom_plugins.get("message", []):
    response_tracking_custom_plugins = restapi_client.upload_custom_plugin_package(
        custom_plugin_name="tracking",
        custom_plugin_file=CUSTOM_PLUGINS_TRACKING_TAR_GZ,
    )

pprint.pprint(response_tracking_custom_plugins)


# Submit transfer learning job for the mobilenet_v2 + yolo network architecture
response_mobilenet_v2_yolo_infer = restapi_client.submit_job(
    workflows_file=WORKFLOWS_TAR_GZ,
    experiment_name=EXPERIMENT_NAME,
    queue="tensorflow_gpu",
    entry_point="infer",
    entry_point_kwargs=" ".join([
        "-P data_dir=/nfs/data/roadsigns/testing",
        "-P batch_size=1",
        f"-P model_name={EXPERIMENT_NAME}_mobilenetv2_yolo",
        "-P model_version=1",
    ]),
)

print("Transfer learning job for MobileNet V2 + YOLO neural network submitted")
print("")
pprint.pprint(response_mobilenet_v2_yolo_infer)

{'collection': 'securingai_custom',
 'modules': ['__init__.py', 'tensorflow.py'],
 'taskPluginName': 'backend_configs'}
{'collection': 'securingai_custom',
 'modules': ['import_keras.py', '__init__.py', 'tensorflow.py'],
 'taskPluginName': 'evaluation'}
{'collection': 'securingai_custom',
 'modules': ['utils.py',
             'output_layers.py',
             'keras_object_detectors.py',
             'feature_extraction_layers.py',
             'metrics.py',
             '__init__.py',
             'finetuning.py',
             'methods.py',
             'losses.py',
             'data.py',
             'keras_classifiers.py'],
 'taskPluginName': 'roadsigns_yolo_estimators'}
{'collection': 'securingai_custom',
 'modules': ['__init__.py', 'mlflow.py'],
 'taskPluginName': 'tracking'}
Transfer learning job for MobileNet V2 + YOLO neural network submitted

{'createdOn': '2022-01-08T16:57:14.915391',
 'dependsOn': None,
 'entryPoint': 'infer',
 'entryPointKwargs': '-P data_dir=/nfs/data/road

In [57]:
MODELS_DIR = Path("models")
YOLO_MOBILENETV2_MODEL_DIR = MODELS_DIR / "roadsigns_yolo_mobilenetv2_yolo" / "1"

PREDICTIONS_DIR = Path("object_detection_predictions")
PREDICTIONS_NPY = PREDICTIONS_DIR / "predictions.npy"
BOXES_NPY = PREDICTIONS_DIR / "boxes.npy"
CLASSES_NPY = PREDICTIONS_DIR / "classes.npy"
NUMS_NPY = PREDICTIONS_DIR / "nums.npy"
SCORES_NPY = PREDICTIONS_DIR / "scores.npy"

In [69]:
CUSTOM_OBJECTS_PKL = YOLO_MOBILENETV2_MODEL_DIR / "custom_objects.cloudpickle"
YOLO_MOBILENETV2 = YOLO_MOBILENETV2_MODEL_DIR / "model"

In [63]:
import tensorflow as tf
import cloudpickle
from mitre.securingai.sdk.utilities.contexts import plugin_dirs

In [72]:
with CUSTOM_OBJECTS_PKL.open("rb") as f, plugin_dirs(["task-plugins"]):
    custom_objects = cloudpickle.load(f)

In [73]:
object_detector = tf.keras.models.load_model(
    YOLO_MOBILENETV2, custom_objects=custom_objects
)

In [77]:
object_detector.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 448, 448, 3)]     0         
_________________________________________________________________
tf.cast (TFOpLambda)         (None, 448, 448, 3)       0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 448, 448, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 448, 448, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 14, 14, 1280)      2257984   
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 1280)        0         
_________________________________________________________________
output_conv_1 (Conv2D)       (None, 7, 7, 1280)        147468

In [30]:
predictions = np.load(PREDICTIONS_NPY)
boxes = np.load(BOXES_NPY)
classes = np.load(CLASSES_NPY)
nums = np.load(NUMS_NPY)
scores = np.load(SCORES_NPY)

In [104]:
(np.expand_dims(predictions[..., 4], -1))[0].max(axis=-1)

array([[-2.0149110e-02,  8.8731766e-02,  1.7855216e-02, -1.2511762e-02,
         1.9298349e-02, -1.3382165e-02, -3.8883962e-02],
       [-1.0208368e-02,  9.5023010e-03,  2.8053893e-02,  3.7045315e-02,
         3.6403477e-02,  1.8372985e-02, -3.1135101e-03],
       [ 9.5464043e-02,  2.9641625e-02,  1.7265847e-01,  2.7345461e-01,
         1.2441099e-01,  5.7044391e-02,  2.7294928e-02],
       [ 7.8367986e-02,  8.4523283e-02,  1.7458114e-01,  4.1367769e-01,
         2.0078835e-01,  3.3028074e-02,  2.2113089e-02],
       [-1.1239146e-02, -1.9665292e-02,  1.2908486e-01,  1.2300809e-01,
         8.4777169e-02,  3.5155457e-02, -5.6423806e-04],
       [-2.2612419e-02,  4.9651463e-02,  5.7586681e-02,  8.6795185e-03,
         3.3643275e-02,  1.1738616e-02, -3.9621215e-02],
       [ 3.5196729e-04, -2.9877417e-02,  2.3215957e-02,  2.4275975e-03,
         7.6973468e-02, -3.1027410e-02, -1.2037779e-02]], dtype=float32)